In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -rm -r -f /tmp/output

Deleted /tmp/output


In [3]:
!hdfs dfs -mkdir /tmp/output

In [4]:
!hdfs dfs -copyFromLocal tbl0.csv /tmp/output/

In [5]:
%%hive

DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>,
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';

LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 10.228 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>,
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.779 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 1.318 seconds


In [21]:
%%hive

DROP TABLE IF EXISTS suma_valores;

CREATE TABLE suma_valores
AS

SELECT c2, SUM(val)
FROM
    (SELECT c2,
            val
     FROM tbl0 LATERAL VIEW explode(c6) tbl0 AS key, val
    ) t0

GROUP BY c2
;
    
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM suma_valores;

DROP TABLE IF EXISTS resultado;
FAILED: SemanticException org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient
CREATE TABLE resultado
AS 
SELECT  a.c1,
        a.c2,
        b.c4
FROM tbl0 a JOIN tbl1 b ON (a.c1 = b.c1);
FAILED: SemanticException org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM resultado;
FAILED: SemanticException org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient


In [18]:
!hdfs dfs -copyToLocal /tmp/output output;

In [19]:
%%hive
DROP TABLE resultado;

DROP TABLE resultado;
OK
Time taken: 0.221 seconds


In [20]:
%hive_quit